In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

ROOT_DIR = os.path.abspath('../..')
sys.path.append(ROOT_DIR)
os.chdir(ROOT_DIR)


# Suppress all future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from transformers import logging

#from tqdm import tqdm
#from functools import partialmethod
#tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

c:\Users\sindo\projetos\neuron\neuron\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from neuron.runtime_logging import start
from neuron.agents import UserAgent
from neuron.agents.non_llm_agents import FeatureImputerAgent, KnowledgeRepresenterAgent, RetrieverAgent
from neuron.agents.llm_agents import PerceiverAgent, AssessorAgent, RecommenderAgent, ExplainableAgent
from neuron.scripts import RoundRobin, RoundRobinManager

#CAPABILITIES
from neuron.capabilities.perceiver import SemanticTemplateFillerCapability
from neuron.capabilities.retriever import DataFrameRetrieverCapability
from neuron.cognitions import ImmediateWorkingMemory

from neuron.components import SequentialComponent, CycleComponent, Pipeline

# Start logging
logging_session_id = start(config={"dbname": "logs.db"})
print("Logging session ID: " + str(logging_session_id))

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


Logging session ID: 508ae59b-5a0e-483a-a80a-88caa263f528


In [3]:
import pandas as pd
df = pd.read_json(r"C:\Users\sindo\projetos\neuron\neuron\data\yelp_academic_dataset_business.json", lines=True, orient='columns', chunksize=1000000)
# read the data 
for business in df:
    business = business.head(100)
    break

business.drop(['business_id', 'is_open', 'hours', 'longitude', 'latitude', 'postal_code', 'state', 'city', 'attributes'], axis=1, inplace=True)
business.rename(columns={'name': 'place_name'}, inplace=True)

In [4]:
# PRESTE ATENÇÃO >>>>> TODO: Agent tem llm_config, o que deveria ser só config.
llm_config={
    "config_list": [
        {
            "client": "groq",
            "model": "llama3-groq-70b-8192-tool-use-preview",
            "api_key": "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"
        }
    ]
}

user_agent = UserAgent(
    name="User Agent",
    llm_config=llm_config,
)

perceiver_agent = PerceiverAgent(llm_config=llm_config)
semantic_template_filler = SemanticTemplateFillerCapability(
    system_message="""
        You will be given a user's interest in a particular item or service. Your task is to classify this interest into a relevant business category and return the classification inside a JSON object. Additionally, you need to extract the following details from the text: address, categories, name, review_count, and stars. If any template field lacks corresponding information in the text, enter np.nan for that field.

        Follow these steps:

        1. Identify the user's interest: Determine the main interest or activity described by the user.
        2. Business Category Matching: Match the user's interest to the most relevant business category.
        3. Extract Additional Information: Extract the address, name, review count, and star rating if they are present in the text.
        4. Return in JSON Format: Output the result in a JSON object with the following fields:
        - "categories": A list with the relevant business category.
        - "address": The address if available or np.nan if not provided.
        - "name": The place name if mentioned or np.nan.
        - "review_count": The number of reviews if mentioned or np.nan.
        - "stars": The star rating if provided or np.nan.

        Example 1:

        Input:
        _I want to get a haircut._

        Expected Output:
        {
        "categories": ["Hair salon"],
        "address": np.nan,
        "name": np.nan,
        "review_count": np.nan,
        "stars": np.nan
        }

        Example 2:

        Input:
        _I want to eat tacos at Maria's, located on 123 Taco Lane, and it has 250 reviews with 4.5 stars._

        Expected Output:
        {
        "categories": ["Mexican restaurant"],
        "address": "123 Taco Lane",
        "name": "Maria's",
        "review_count": "250",
        "stars": "4.5"
        }
    """, 
    llm_config=llm_config
    )
semantic_template_filler.add_to_agent(perceiver_agent)


feature_imputer_agent =  FeatureImputerAgent(
    llm_config = {
    "config_list": [{
        "client": "Begreat",
        "model": "distilgpt2",
        "model_dir": r"C:\Users\sindo\projetos\neuron\neuron\models\be_great_yelp",
    }]},
)

retriever_agent = RetrieverAgent(llm_config=llm_config)
data_frame_retriever_capability = DataFrameRetrieverCapability(
    dataset=business, 
    columns = ["categories"],
    config= {
        "model": "gpt2-medium",
        "model_dir": r"C:\Users\sindo\projetos\neuron\neuron\models\erasmo_yelp_gpt2-medium_60_True"
    }
)
data_frame_retriever_capability.add_to_agent(retriever_agent)

assessor_agent = AssessorAgent(llm_config=llm_config)
shared_memory = ImmediateWorkingMemory(neurons=[user_agent])
shared_memory.add_to_agent(assessor_agent)

recommender_agent = RecommenderAgent(llm_config=llm_config)
explainer_agent = ExplainableAgent(llm_config=llm_config)


c:\Users\sindo\projetos\neuron\neuron\.venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sindo\.cache\huggingface\hub\models--distilgpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\sindo\projetos\neuron\neuron\.venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `h

In [5]:
sequential1 = SequentialComponent(agents=[perceiver_agent])
cycle = CycleComponent(agents=[feature_imputer_agent, retriever_agent, assessor_agent], repetitions=1)
#sequential2 = SequentialComponent(agents=[recommender_agent, explainer_agent])

pipeline = Pipeline([sequential1, cycle])

In [6]:
user_agent.initiate_chat(
    pipeline, message="Italian restaurant",
)

User Agent (to perceiver_agent):

Italian restaurant

--------------------------------------------------------------------------------
perceiver_agent (to feature_imputer):

{
  "categories": ["Italian restaurant"],
  "address": np.nan,
  "name": np.nan,
  "review_count": np.nan,
  "stars": np.nan
}

--------------------------------------------------------------------------------


  0%|          | 0/1 [00:14<?, ?it/s]


RuntimeError: BeGreat exception occurred: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [15]:
assessor_agent._oai_messages

defaultdict(list,
            {<neuron.agents.non_llm_agents.retriever_agent.RetrieverAgent at 0x1dc8e953ac0>: [{'content': 'Item 31:\n- Address: 13071 Yonge Street\n- Cuisine: Persian/Iranian, Turkish, Middle Eastern\n- Category: Restaurants\n- Specialization: Kebab\n- Name: Apadana Restaurant\n- Rating: 3\n- Average Rating: 3.0\n\nItem 38:\n- Address: 1300 N 12th St, Ste 605\n- Category: Doctors, Health & Medical\n- Specialization: Preventive Medicine, Medical Centers\n- Name: Banner - University Family Medicine Center\n- Rating: 3\n- Average Rating: 3.0\n\nItem 58:\n- Address: 2490 Rue Notre-Dame O\n- Category: Nightlife, Arts & Entertainment\n- Specialization: Performing Arts, Music Venues\n- Name: Théâtre Corona\n- Rating: 18\n- Average Rating: 4.0',
               'name': 'retriever_agent',
               'role': 'user'}],
             <neuron.agents.user_agent.UserAgent at 0x1dc8e953310>: [{'content': 'No',
               'name': 'assessor_agent',
               'role': 'assista

In [16]:
user_agent._oai_messages

defaultdict(list,
            {<neuron.components.pipeline.Pipeline at 0x1dc8ebc1090>: [{'content': 'Italian restaurant',
               'name': 'User Agent',
               'role': 'assistant'}],
             <neuron.agents.llm_agents.perceiver_agent.PerceiverAgent at 0x1dc8e953a90>: [{'content': 'Italian restaurant',
               'name': 'User Agent',
               'role': 'assistant'}],
             <neuron.agents.llm_agents.assessor_agent.AssessorAgent at 0x1dd066f5a20>: [{'content': 'No',
               'name': 'assessor_agent',
               'role': 'user'}]})

In [17]:
stop here

SyntaxError: invalid syntax (4067800170.py, line 1)

In [ ]:
data_frame_retriever_capability._dataset.head(200)

In [ ]:
data_frame_retriever_capability.retrieve_similar("Occupational Therapy, Health & Medical, Massage", 5)

### Components test

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

ROOT_DIR = os.path.abspath('../..')
sys.path.append(ROOT_DIR)
os.chdir(ROOT_DIR)


# Suppress all future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from transformers import logging

#from tqdm import tqdm
#from functools import partialmethod
#tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

In [2]:
from neuron.components import SequentialComponent, CycleComponent, Pipeline
from neuron.agents import AssistantAgent
from neuron.agents import UserAgent

In [3]:
llm_config={
    "config_list": [
        {
            "client": "groq",
            "model": "llama3-groq-70b-8192-tool-use-preview",
            "api_key": "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"
        }
    ]
}

user_agent = UserAgent(name="User", llm_config=llm_config)

comomn_sm = "Dado um numero fornecido pelo usuario, vc deve adicionar mais um a esse numero. Seu output é somente o numero somado, sem texto adicional. Considere a ultima mensagem do user."
a1 = AssistantAgent(name="Assistante 1", llm_config=llm_config, system_message=comomn_sm)

a2 = AssistantAgent(name="Assistante 2", llm_config=llm_config, system_message=comomn_sm)
a3 = AssistantAgent(name="Assistante 3", llm_config=llm_config, system_message=comomn_sm)

a4 = AssistantAgent(name="Assistante 4", llm_config=llm_config, system_message=comomn_sm)
a5 = AssistantAgent(name="Assistante 5", llm_config=llm_config, system_message=comomn_sm)

seq1 = SequentialComponent(agents=[a1])
cycle = CycleComponent(agents=[a2, a3, a4], repetitions=3)
seq2 = SequentialComponent(agents=[a4])

pipeline = Pipeline([seq1, cycle, seq2])

In [ ]:
user_agent.initiate_chat(
    pipeline, message="Comece de 0"
)